# Generating Useful Wikidata Files

This notebook generates files that contain derived data that is useful in many applications. The input to the notebook is the full Wikidata or a subset of Wikidata. It also works for arbutrary KGs as long as they follow the representation requirements of Wikidata:

- the *instance of* relation is represented using the `P31` property
- the *subclass of* relation is represented using the `P279` property
- all properties declare a datatype, and the data types must be one of the datatypes in Wikidata.

Inputs:

- `claims_file`: contains all statements, which consist of edges `node1/label/node2` where `label` is a property in Wikidata (e.g., sitelinks, labels, aliases and description are not in the claims file.
- `item_file`: the subset of the `claims_file` consistin of edges for property of data type `wikibase-item`
- `label_file`, `alias_file` and `description_file` containing labels, aliases and descriptions. It is assume that these files contain the labels, aliases and descriptions of all nodes appearing in the claims file. Users may provide these files for specific languages only.

Outputs:

- **Instance of (P31):** `derived.P31.tsv.gz` contains all the `instance of (P31)` edges present in the claims file.
- **Subclass of (P279):** `derived.P279.tsv.gz` contains all the `subclass of (P279)` edges present in the claims file.
- **Is A (isa):** `derived.isa.tsv.gz` contains edges `node`isa/node2` where either `node1/P31/node2` or `node1/P279/node2`
- **Closure of subclass of (P279star):** `derived.P279star.tsv.gz` contains edges `node1/P279star/node2` where `node2` is reachable from `node1` via zero or more hops using the `P279` property. Note that for example, `Q44/P279star/Q44`. An example when this file is useful is when you want to find all the instance of a class, including instances of subclasses of the given class.
- **In/out degrees:** `metadata.out_degree.tsv.gz` contains the out degree of every node, and `metadata.in_degree.tsv.gz` contains the in degree of every node.
- **Pagerank:** outputs page rank on the directed graph in `metadata.pagerank.directed.tsv.gz` and page rank of the directed graph in `metadata.pagerank.undirected.tsv.gz`.

### Batch Invocation
Example batch command. The second argument is a notebook where the output will be stored. You can load it to see progress.

```
papermill Wikidata\ Useful\ Files.ipynb useful-files.out.ipynb \
-p claims_file /Volumes/GoogleDrive/Shared\ drives/KGTK-public-graphs/wikidata-20200803-v4/all.tsv.gz \
-p label_file /Volumes/GoogleDrive/Shared\ drives/KGTK-public-graphs/wikidata-20200803-v4/part.label.en.tsv.gz \
-p item_file /Volumes/GoogleDrive/Shared\ drives/KGTK-public-graphs/wikidata-20200803-v4/part.wikibase-item.tsv.gz \
-p property_item_file = /Volumes/GoogleDrive/Shared\ drives/KGTK-public-graphs/wikidata-20200803-v4/part.property.wikibase-item.tsv.gz \
-p output_path <local folder> \
-p output_folder useful_files_v4 \
-p temp_folder temp.useful_files_v4 \
-p delete_database no 
-p languages es,ru,zh-cn
```

In [2]:
# Parameters

# Folder on local machine where to create the output and temporary folders
output_path = "/data/amandeep/wikidata-20210215"

# The names of the output and temporary folders
output_folder = "useful_wikidata_files"
temp_folder = "temp.useful_wikidata_files"

# The location of input files
wiki_root_folder = "/data/amandeep/wikidata-20210215/"
claims_file = "claims.tsv.gz"
label_file = "labels.en.tsv.gz"
alias_file = "aliases.en.tsv.gz"
description_file = "descriptions.en.tsv.gz"
item_file = "claims.wikibase-item.tsv.gz"

label_all = "labels.tsv.gz"
alias_all = "aliases.tsv.gz"
description_all = "descriptions.tsv.gz"

# Location of the cache database for kypher
cache_path = "/data/amandeep/wikidata-20210215/temp.useful_wikidata_files"

# Whether to delete the cache database
delete_database = False

# Whether to compute pagerank as it may not run on the laptop
compute_pagerank = True
languages = 'ru,es,zh-cn,de,it,nl,pl,fr,pt,sv'

In [3]:
# Parameters
output_path = "/data/hrathod/output/subset/novartis"
output_folder = "useful_files"
temp_folder = "temp.useful_files"
wiki_root_folder = "/data/hrathod/output/subset/novartis/parts/"
cache_path = "/data/hrathod/output/subset/novartis/temp.useful_files"
languages = "en"
compute_pagerank = True
delete_database = False


In [4]:
languages = languages.split(',')

In [5]:
import io
import os
import subprocess
import sys

import numpy as np
import pandas as pd

import altair as alt

## Set up environment and folders to store the files

- `OUT` folder where the output files go
- `TEMP` folder to keep temporary files , including the database
- `kgtk` shortcut to invoke the kgtk software
- `kypher` shortcut to invoke `kgtk query with the cache database
- `CLAIMS` the `all.tsv` file of wikidata that contains all edges except label/alias/description
- `LABELS` the file with the English labels
- `ITEMS` the wikibase-item file (currently does not include node1 that are properties so for now we need the net file
- `STORE` location of the cache file

In [6]:
if cache_path:
    os.environ['STORE'] = "{}/wikidata.sqlite3.db".format(cache_path)
else:
    os.environ['STORE'] = "{}/{}/wikidata.sqlite3.db".format(output_path, temp_folder)
os.environ['OUT'] = "{}/{}".format(output_path, output_folder)
os.environ['TEMP'] = "{}/{}".format(output_path, temp_folder)
os.environ['kgtk'] = "kgtk"
os.environ['kgtk'] = "kgtk --debug"
os.environ['kypher'] = "kgtk --debug query --graph-cache " + os.environ['STORE']
os.environ['CLAIMS'] = wiki_root_folder + claims_file
os.environ['LABELS'] = wiki_root_folder + label_file
os.environ['ALIASES'] = wiki_root_folder + alias_file
os.environ['DESCRIPTIONS'] = wiki_root_folder + description_file
os.environ['ITEMS'] = wiki_root_folder + item_file

Go to the output directory and create the subfolders for the output files and the temporary files

In [7]:
cd $output_path

/data/hrathod/output/subset/novartis


In [8]:
!mkdir -p $OUT
!mkdir -p $TEMP

Clean up the database before we start

In [9]:
if delete_database:
    print("Deleteddatabase") 
    !rm $STORE

### Preview the input files

It is always a good practice to peek a the files to make sure the column headings are what we expect

In [12]:
!$kypher -i "$CLAIMS" --limit 10 | col 

[2021-04-19 09:37:19 sqlstore]: IMPORT graph directly into table graph_1 from /data/hrathod/output/subset/novartis/parts/claims.tsv.gz ...
[2021-04-19 09:45:32 query]: SQL Translation:
---------------------------------------------
  SELECT *
     FROM graph_1 AS graph_1_c1
     LIMIT ?
  PARAS: [10]
---------------------------------------------
id	node1	label	node2	node2;wikidatatype	rank
P10-P1628-32b85d-7927ece6-0	P10	P1628	"http://www.w3.org/2006/vcard/ns#Video" url	normal
P10-P1628-acf60d-b8950832-0	P10	P1628	"https://schema.org/video"	url	normal
P10-P1629-Q34508-bcc39400-0	P10	P1629	Q34508	wikibase-item	normal
P10-P1659-P1651-c4068028-0	P10	P1659	P1651	wikibase-property	normal
P10-P1659-P18-5e4b9c4f-0	P10	P1659	P18	wikibase-property	normal
P10-P1659-P4238-d21d1ac0-0	P10	P1659	P4238	wikibase-property	normal
P10-P1659-P51-86aca4c5-0	P10	P1659	P51	wikibase-property	normal
P10-P1855-Q15075950-7eff6d65-0	P10	P1855	Q15075950	wikibase-item	normal
P10-P1855-Q4504-a69d2c73-0	P10	P1855	Q450

Force creation of the index on the label column

In [13]:
!$kypher -i "$CLAIMS" -o - \
--match '(i)-[:P31]->(c)' \
--limit 5 \
| column -t -s $'\t' 

[2021-04-19 09:45:44 query]: SQL Translation:
---------------------------------------------
  SELECT *
     FROM graph_1 AS graph_1_c1
     WHERE graph_1_c1."label"=?
     LIMIT ?
  PARAS: ['P31', 5]
---------------------------------------------
[2021-04-19 09:45:44 sqlstore]: CREATE INDEX on table graph_1 column label ...
[2021-04-19 09:51:06 sqlstore]: ANALYZE INDEX on table graph_1 column label ...
id                              node1  label  node2      node2;wikidatatype  rank
P10-P31-Q18610173-85ef4d24-0    P10    P31    Q18610173  wikibase-item       normal
P1000-P31-Q18608871-093affb5-0  P1000  P31    Q18608871  wikibase-item       normal
P1001-P31-Q15720608-deeedec9-0  P1001  P31    Q15720608  wikibase-item       normal
P1001-P31-Q22984026-8beb0cfe-0  P1001  P31    Q22984026  wikibase-item       normal
P1001-P31-Q22997934-1e5b1a96-0  P1001  P31    Q22997934  wikibase-item       normal


Force creation of the index on the node2 column

In [14]:
!$kypher -i "$CLAIMS" -o - \
--match '(i)-[r]->(:Q5)' \
--limit 5 \
| column -t -s $'\t' 

[2021-04-19 09:51:29 query]: SQL Translation:
---------------------------------------------
  SELECT *
     FROM graph_1 AS graph_1_c1
     WHERE graph_1_c1."node2"=?
     LIMIT ?
  PARAS: ['Q5', 5]
---------------------------------------------
[2021-04-19 09:51:29 sqlstore]: CREATE INDEX on table graph_1 column node2 ...
[2021-04-19 09:59:05 sqlstore]: ANALYZE INDEX on table graph_1 column node2 ...
id                         node1  label  node2  node2;wikidatatype  rank
P1424-P1855-Q5-47bdcd17-0  P1424  P1855  Q5     wikibase-item       normal
P1552-P1855-Q5-53b667e4-0  P1552  P1855  Q5     wikibase-item       normal
P1963-P1855-Q5-1ba43aca-0  P1963  P1855  Q5     wikibase-item       normal
P3055-P1629-Q5-fb63cfeb-0  P3055  P1629  Q5     wikibase-item       normal
P5869-P1855-Q5-3a19317f-0  P5869  P1855  Q5     wikibase-item       normal


### Count the number of edges

Counting takes a long time

In [15]:
!$kypher -i "$CLAIMS" \
--match '()-[r]->()' \
--return 'count(r) as count' \
--limit 10

[2021-04-19 09:59:39 query]: SQL Translation:
---------------------------------------------
  SELECT count(graph_1_c1."id") "_aLias.count"
     FROM graph_1 AS graph_1_c1
     LIMIT ?
  PARAS: [10]
---------------------------------------------
count
293787649


### Get labels, aliases and descriptions for other languages

In [16]:
for lang in languages:
    cmd = f"kgtk --debug query --graph-cache {os.environ['STORE']} -i {wiki_root_folder}{label_all} -o {output_path}/{output_folder}/labels.{lang}.tsv.gz --match '(n1)-[l:label]->(n2)' --where 'n2.kgtk_lqstring_lang_suffix = \"{lang}\"' --return 'n1, l.label, n2, l.id' "
    !{cmd}

[2021-04-19 10:00:14 sqlstore]: IMPORT graph directly into table graph_2 from /data/hrathod/output/subset/novartis/parts/labels.tsv.gz ...
[2021-04-19 10:01:02 query]: SQL Translation:
---------------------------------------------
  SELECT graph_2_c1."node1", graph_2_c1."label", graph_2_c1."node2", graph_2_c1."id"
     FROM graph_2 AS graph_2_c1
     WHERE graph_2_c1."label"=?
     AND (kgtk_lqstring_lang_suffix(graph_2_c1."node2") = ?)
  PARAS: ['label', 'en']
---------------------------------------------
[2021-04-19 10:01:02 sqlstore]: CREATE INDEX on table graph_2 column label ...
[2021-04-19 10:01:15 sqlstore]: ANALYZE INDEX on table graph_2 column label ...


In [17]:
for lang in languages:
    cmd = f"kgtk --debug query --graph-cache {os.environ['STORE']} -i {wiki_root_folder}{alias_all} -o {output_path}/{output_folder}/aliases.{lang}.tsv.gz --match '(n1)-[l:alias]->(n2)' --where 'n2.kgtk_lqstring_lang_suffix = \"{lang}\"' --return 'n1, l.label, n2, l.id' "
    !{cmd}

[2021-04-19 10:06:22 sqlstore]: IMPORT graph directly into table graph_3 from /data/hrathod/output/subset/novartis/parts/aliases.tsv.gz ...
[2021-04-19 10:06:32 query]: SQL Translation:
---------------------------------------------
  SELECT graph_3_c1."node1", graph_3_c1."label", graph_3_c1."node2", graph_3_c1."id"
     FROM graph_3 AS graph_3_c1
     WHERE graph_3_c1."label"=?
     AND (kgtk_lqstring_lang_suffix(graph_3_c1."node2") = ?)
  PARAS: ['alias', 'en']
---------------------------------------------
[2021-04-19 10:06:32 sqlstore]: CREATE INDEX on table graph_3 column label ...
[2021-04-19 10:06:35 sqlstore]: ANALYZE INDEX on table graph_3 column label ...


In [18]:
for lang in languages:
    cmd = f"kgtk --debug query --graph-cache {os.environ['STORE']} -i {wiki_root_folder}{description_all} -o {output_path}/{output_folder}/descriptions.{lang}.tsv.gz --match '(n1)-[l:description]->(n2)' --where 'n2.kgtk_lqstring_lang_suffix = \"{lang}\"' --return 'n1, l.label, n2, l.id' "
    !{cmd}

[2021-04-19 10:07:50 sqlstore]: IMPORT graph directly into table graph_4 from /data/hrathod/output/subset/novartis/parts/descriptions.tsv.gz ...
[2021-04-19 10:08:29 query]: SQL Translation:
---------------------------------------------
  SELECT graph_4_c1."node1", graph_4_c1."label", graph_4_c1."node2", graph_4_c1."id"
     FROM graph_4 AS graph_4_c1
     WHERE graph_4_c1."label"=?
     AND (kgtk_lqstring_lang_suffix(graph_4_c1."node2") = ?)
  PARAS: ['description', 'en']
---------------------------------------------
[2021-04-19 10:08:29 sqlstore]: CREATE INDEX on table graph_4 column label ...
[2021-04-19 10:08:40 sqlstore]: ANALYZE INDEX on table graph_4 column label ...


### Create the P31 and P279 files

Create the `P31` file

In [19]:
!$kypher -i "$CLAIMS" -o $OUT/derived.P31.tsv.gz \
--match '(n1)-[l:P31]->(n2)' \
--return 'l, n1, l.label, n2' 

[2021-04-19 10:11:48 query]: SQL Translation:
---------------------------------------------
  SELECT graph_1_c1."id", graph_1_c1."node1", graph_1_c1."label", graph_1_c1."node2"
     FROM graph_1 AS graph_1_c1
     WHERE graph_1_c1."label"=?
  PARAS: ['P31']
---------------------------------------------


Create the P279 file

In [18]:
!zcat $OUT/derived.P31.tsv.gz | head | col

id	node1	label	node2
P10-P31-Q18610173-85ef4d24-0	P10	P31	Q18610173
P1000-P31-Q18608871-093affb5-0	P1000	P31	Q18608871
P1001-P31-Q15720608-deeedec9-0	P1001	P31	Q15720608
P1001-P31-Q22984026-8beb0cfe-0	P1001	P31	Q22984026

gzip: P1001-P31-Q22997934-1e5b1a96-0	P1001	P31	Q22997934
stdout: Broken pipe
P1001-P31-Q61719275-0ccc11a5-0	P1001	P31	Q61719275
P1001-P31-Q70564278-b92b04ba-0	P1001	P31	Q70564278
P1002-P31-Q22963600-b3a47587-0	P1002	P31	Q22963600
P1003-P31-Q19595382-152d2cdd-0	P1003	P31	Q19595382


In [21]:
!$kypher -i "$CLAIMS" -o $OUT/derived.P279.tsv.gz \
    --match '(n1)-[l:P279]->(n2)' \
    --return 'l, n1, l.label, n2' 

[2021-04-19 10:19:56 query]: SQL Translation:
---------------------------------------------
  SELECT graph_1_c1."id", graph_1_c1."node1", graph_1_c1."label", graph_1_c1."node2"
     FROM graph_1 AS graph_1_c1
     WHERE graph_1_c1."label"=?
  PARAS: ['P279']
---------------------------------------------


### Create the file that contains all nodes reachable via P279 starting from a node2 in P31 or a node1 in P279

First compute the roots

In [22]:
!$kypher -i $OUT/derived.P279.tsv.gz -o $TEMP/P279.n1.tsv.gz \
--match '(n1)-[l]->()' \
--return 'n1 as id' 

[2021-04-19 10:20:48 sqlstore]: IMPORT graph directly into table graph_5 from /data/hrathod/output/subset/novartis/useful_files/derived.P279.tsv.gz ...
[2021-04-19 10:21:04 query]: SQL Translation:
---------------------------------------------
  SELECT graph_5_c1."node1" "_aLias.id"
     FROM graph_5 AS graph_5_c1
  PARAS: []
---------------------------------------------


In [23]:
!$kypher -i $OUT/derived.P31.tsv.gz -o $TEMP/P31.n2.tsv.gz \
--match '()-[l]->(n2)' \
--return 'n2 as id' 

[2021-04-19 10:21:23 sqlstore]: IMPORT graph directly into table graph_6 from /data/hrathod/output/subset/novartis/useful_files/derived.P31.tsv.gz ...
[2021-04-19 10:22:06 query]: SQL Translation:
---------------------------------------------
  SELECT graph_6_c1."node2" "_aLias.id"
     FROM graph_6 AS graph_6_c1
  PARAS: []
---------------------------------------------


In [24]:
!$kgtk cat --mode NONE -i $TEMP/P31.n2.tsv.gz $TEMP/P279.n1.tsv.gz \
| gzip > $TEMP/P279.roots.1.tsv.gz

In [25]:
!$kgtk sort --mode NONE --column id -i $TEMP/P279.roots.1.tsv.gz \
| gzip > $TEMP/P279.roots.2.tsv.gz

We have lots of duplicates

In [26]:
!zcat < $TEMP/P279.roots.2.tsv.gz | head

id
Q100000030
Q1000017
Q1000032
Q1000032
Q1000039
Q100004761
Q100004761
Q100004761
Q1000064

gzip: stdout: Broken pipe


In [27]:
!$kgtk compact -i $TEMP/P279.roots.2.tsv.gz --mode NONE \
    --presorted \
    --columns id \
    -o $TEMP/P279.roots.tsv

Now we can invoke the reachable-nodes command

In [28]:
!$kgtk reachable-nodes \
    --rootfile $TEMP/P279.roots.tsv \
    --selflink \
    -i $OUT/derived.P279.tsv.gz \
| gzip > $TEMP/P279.reachable.tsv.gz

In [29]:
!zcat < $TEMP/P279.reachable.tsv.gz | head | col

node1	label	node2
Q100000030	reachable	Q100000030
Q100000030	reachable	Q14748
Q100000030	reachable	Q14745
Q100000030	reachable	Q1357761
Q100000030	reachable	Q223557
Q100000030	reachable	Q4406616
Q100000030	reachable	Q488383
Q100000030	reachable	Q35120

gzip: Q100000030	reachable	Q2424752
stdout: Broken pipe


The reachable-nodes command produces edges labeled `reachable`, so we need one command to rename them.

In [30]:
!$kypher -i $TEMP/P279.reachable.tsv.gz -o $TEMP/P279star.1.tsv.gz \
--match '(n1)-[]->(n2)' \
--return 'n1, "P279star" as label, n2 as node2' 

[2021-04-19 11:41:33 sqlstore]: IMPORT graph directly into table graph_7 from /data/hrathod/output/subset/novartis/temp.useful_files/P279.reachable.tsv.gz ...
[2021-04-19 11:44:23 query]: SQL Translation:
---------------------------------------------
  SELECT graph_7_c1."node1", ? "_aLias.label", graph_7_c1."node2" "_aLias.node2"
     FROM graph_7 AS graph_7_c1
  PARAS: ['P279star']
---------------------------------------------


Now we can concatenate these files to produce the final output

In [31]:
!$kgtk sort -i $TEMP/P279star.1.tsv.gz -o $TEMP/P279star.2.tsv.gz

Make sure there are no duplicates

In [32]:
!$kgtk compact --presorted -i $TEMP/P279star.2.tsv.gz -o $TEMP/P279star.3.tsv.gz

Add ids

In [33]:
!$kgtk add-id --id-style wikidata -i $TEMP/P279star.3.tsv.gz -o $OUT/derived.P279star.tsv.gz

In [34]:
!zcat < $OUT/derived.P279star.tsv.gz | head | col

node1	label	node2	id
Q100000030	P279star	Q100000030	Q100000030-P279star-Q100000030-0000
Q100000030	P279star	Q1357761	Q100000030-P279star-Q1357761-0000
Q100000030	P279star	Q14745	Q100000030-P279star-Q14745-0000
Q100000030	P279star	Q14748	Q100000030-P279star-Q14748-0000
Q100000030	P279star	Q15401930	Q100000030-P279star-Q15401930-0000
Q100000030	P279star	Q15621286	Q100000030-P279star-Q15621286-0000
Q100000030	P279star	Q16686448	Q100000030-P279star-Q16686448-0000
Q100000030	P279star	Q17537576	Q100000030-P279star-Q17537576-0000
Q100000030	P279star	Q223557 Q100000030-P279star-Q223557-0000

gzip: stdout: Broken pipe


This is how we would do the typical `?item P31/P279* ?class` in Kypher. 
The example shows how to get all the counts of instances of subclasses of city (Q515).

In [35]:
!$kypher -i $OUT/derived.P31.tsv.gz -i $OUT/derived.P279star.tsv.gz -i "$LABELS" \
--match 'P31: (n1)-[:P31]->(c), P279star: (c)-[]->(:Q515), label: (n1)-[:label]->(label), label: (c)-[:label]->(c_label)' \
--return 'distinct c as class, count(c) as count, c_label as `class name`, n1 as instance, label as `label`' \
--order-by 'count(c) desc, c, n1' \
--limit 10 \
| col

[2021-04-19 14:46:22 sqlstore]: IMPORT graph directly into table graph_8 from /data/hrathod/output/subset/novartis/useful_files/derived.P279star.tsv.gz ...
[2021-04-19 14:52:03 sqlstore]: IMPORT graph directly into table graph_9 from /data/hrathod/output/subset/novartis/parts/labels.en.tsv.gz ...
[2021-04-19 14:52:52 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_6_c1."node2" "_aLias.class", count(graph_6_c1."node2") "_aLias.count", graph_9_c4."node2" "_aLias.class name", graph_6_c1."node1" "_aLias.instance", graph_9_c3."node2" "_aLias.label"
     FROM graph_6 AS graph_6_c1, graph_8 AS graph_8_c2, graph_9 AS graph_9_c3, graph_9 AS graph_9_c4
     WHERE graph_6_c1."label"=?
     AND graph_8_c2."node2"=?
     AND graph_9_c3."label"=?
     AND graph_9_c4."label"=?
     AND graph_6_c1."node1"=graph_9_c3."node1"
     AND graph_6_c1."node2"=graph_8_c2."node1"
     AND graph_8_c2."node1"=graph_9_c4."node1"
     GROUP BY "_aLias.class"
     ORDER

Illustrate that it is indeed `P279*`

In [36]:
!$kypher -i $OUT/derived.P31.tsv.gz -i $OUT/derived.P279star.tsv.gz -i "$LABELS" \
--match 'P31: (n1)-[:P31]->(c), P279star: (c)-[]->(:Q63440326), label: (n1)-[:label]->(label), label: (c)-[:label]->(c_label)' \
--return 'distinct c as class, c_label as `class name`, n1 as instance, label as `label`' \
--order-by 'c, n1' \
--limit 10 \
| col 

[2021-04-19 15:08:03 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_6_c1."node2" "_aLias.class", graph_9_c4."node2" "_aLias.class name", graph_9_c3."node1" "_aLias.instance", graph_9_c3."node2" "_aLias.label"
     FROM graph_6 AS graph_6_c1, graph_8 AS graph_8_c2, graph_9 AS graph_9_c3, graph_9 AS graph_9_c4
     WHERE graph_6_c1."label"=?
     AND graph_8_c2."node2"=?
     AND graph_9_c3."label"=?
     AND graph_9_c4."label"=?
     AND graph_6_c1."node1"=graph_9_c3."node1"
     AND graph_6_c1."node2"=graph_8_c2."node1"
     AND graph_8_c2."node1"=graph_9_c4."node1"
     ORDER BY graph_6_c1."node2" ASC, graph_9_c3."node1" ASC
     LIMIT ?
  PARAS: ['P31', 'Q63440326', 'label', 'label', 10]
---------------------------------------------
class	class name	instance	label


Test that `P279star` is indeed star

In [37]:
!$kypher -i $OUT/derived.P279star.tsv.gz \
--match '(n1:Q44)-[:P279star]->(n2:Q44)'

[2021-04-19 15:08:04 query]: SQL Translation:
---------------------------------------------
  SELECT *
     FROM graph_8 AS graph_8_c1
     WHERE graph_8_c1."label"=?
     AND graph_8_c1."node1"=?
     AND graph_8_c1."node2"=?
  PARAS: ['P279star', 'Q44', 'Q44']
---------------------------------------------
[2021-04-19 15:08:04 sqlstore]: CREATE INDEX on table graph_8 column label ...
[2021-04-19 15:10:44 sqlstore]: ANALYZE INDEX on table graph_8 column label ...
node1	label	node2	id
Q44	P279star	Q44	Q44-P279star-Q44-0000


### Create a file to do generalized Is-A queries
The idea is that `(n1)-[:isa]->(n2)` when `(n1)-[:P31]->(n2)` or `(n1)-[:P279]->(n2)`

We do this by concatenating the files and renaming the relation

In [38]:
!$kgtk cat -i $OUT/derived.P31.tsv.gz $OUT/derived.P279.tsv.gz \
-o $TEMP/isa.1.tsv.gz

In [39]:
!$kypher -i $TEMP/isa.1.tsv.gz -o $OUT/derived.isa.tsv.gz \
--match '(n1)-[]->(n2)' \
--return 'n1, "isa" as label, n2' 

[2021-04-19 15:39:25 sqlstore]: IMPORT graph directly into table graph_10 from /data/hrathod/output/subset/novartis/temp.useful_files/isa.1.tsv.gz ...
[2021-04-19 15:42:53 query]: SQL Translation:
---------------------------------------------
  SELECT graph_10_c1."node1", ? "_aLias.label", graph_10_c1."node2"
     FROM graph_10 AS graph_10_c1
  PARAS: ['isa']
---------------------------------------------


Example of how to use the `isa` relation

In [40]:
!$kypher -i $OUT/derived.isa.tsv.gz -i $OUT/derived.P279star.tsv.gz -i "$LABELS" -o - \
--match 'isa: (n1)-[l:isa]->(c), P279star: (c)-[]->(:Q44), label: (n1)-[:label]->(label)' \
--return 'distinct n1, l.label, "Q44" as node2, label as n1_label' \
--limit 10 \
| col

[2021-04-19 15:49:17 sqlstore]: IMPORT graph directly into table graph_11 from /data/hrathod/output/subset/novartis/useful_files/derived.isa.tsv.gz ...
[2021-04-19 15:49:49 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_9_c3."node1", graph_11_c1."label", ? "_aLias.node2", graph_9_c3."node2" "_aLias.n1_label"
     FROM graph_11 AS graph_11_c1, graph_8 AS graph_8_c2, graph_9 AS graph_9_c3
     WHERE graph_11_c1."label"=?
     AND graph_8_c2."node2"=?
     AND graph_9_c3."label"=?
     AND graph_11_c1."node1"=graph_9_c3."node1"
     AND graph_11_c1."node2"=graph_8_c2."node1"
     LIMIT ?
  PARAS: ['Q44', 'isa', 'Q44', 'label', 10]
---------------------------------------------
[2021-04-19 15:49:49 sqlstore]: CREATE INDEX on table graph_11 column label ...
[2021-04-19 15:50:26 sqlstore]: ANALYZE INDEX on table graph_11 column label ...
[2021-04-19 15:50:45 sqlstore]: CREATE INDEX on table graph_11 column node2 ...
[2021-04-19 15:54:09 sqlstore

### Create files with `isa/P279* and P31/P279*` 
This file is useful to find all nodes that are below a q-node via P279 or isa.

> These files are very large and take many hours to compute

In [42]:
os.environ['P279STAR'] = f"{os.environ['OUT']}/derived.P279star.tsv.gz"
os.environ['ISA'] = f"{os.environ['OUT']}/derived.isa.tsv.gz"

In [43]:
!ls -l $CLAIMS
!ls -l $P279STAR
!ls -l $ISA

-rw-r--r-- 1 hrathod div22 6197025530 Apr 19 00:38 /data/hrathod/output/subset/novartis/parts/claims.tsv.gz
-rw-r--r-- 1 hrathod div22 500895934 Apr 19 14:45 /data/hrathod/output/subset/novartis/useful_files/derived.P279star.tsv.gz
-rw-r--r-- 1 hrathod div22 113117957 Apr 19 15:49 /data/hrathod/output/subset/novartis/useful_files/derived.isa.tsv.gz


In [44]:
!$kypher -i "$P279STAR" -i "$ISA"  \
--match '\
  isa: (n1)-[]->(n2), \
  P279star: (n2)-[]->(n3)' \
--return 'distinct n1 as node1, "isa_star" as label, n3 as node2' \
-o "$TEMP"/derived.isastar_1.tsv.gz

[2021-04-19 16:04:40 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_11_c1."node1" "_aLias.node1", ? "_aLias.label", graph_8_c2."node2" "_aLias.node2"
     FROM graph_11 AS graph_11_c1, graph_8 AS graph_8_c2
     WHERE graph_11_c1."node2"=graph_8_c2."node1"
  PARAS: ['isa_star']
---------------------------------------------


Now add ids and sort it

In [57]:
!$kgtk add-id --id-style wikidata -i "$TEMP"/derived.isastar_1.tsv.gz \
/ sort -o "$OUT"/derived.isastar.tsv.gz

It is very big

In [58]:
!zcat < "$OUT"/derived.isastar.tsv.gz | wc

1124675882 4498703528 62256821797


Also calculate the same file by for P31/P279*

In [59]:
!$kypher -i "$CLAIMS" -i "$P279STAR" \
--match '\
  claims: (n1)-[:P31]->(n2), \
  P279star: (n2)-[]->(n3)' \
--return 'distinct n1 as node1, "P31P279star" as label, n3 as node2' \
-o "$TEMP"/derived.P31P279star.gz

[2021-04-20 04:01:17 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_1_c1."node1" "_aLias.node1", ? "_aLias.label", graph_8_c2."node2" "_aLias.node2"
     FROM graph_1 AS graph_1_c1, graph_8 AS graph_8_c2
     WHERE graph_1_c1."label"=?
     AND graph_1_c1."node2"=graph_8_c2."node1"
  PARAS: ['P31P279star', 'P31']
---------------------------------------------


Add ids and sort it

In [60]:
!$kgtk add-id --id-style wikidata -i "$TEMP"/derived.P31P279star.gz \
/ sort -o "$OUT"/derived.P31P279star.tsv.gz

It is also very big

In [61]:
!zcat < "$OUT"/derived.P31P279star.tsv.gz | wc

1108414952 4433659808 68009062735


## Compute pagerank

Now compute pagerank. These commands will exceed 16GB memory for graphs containing over 25 million nodes.

In [62]:
if compute_pagerank:
    !$kgtk graph-statistics -i "$ITEMS" -o $OUT/metadata.pagerank.directed.tsv.gz \
    --page-rank-property directed_pagerank \
    --pagerank --statistics-only \
    --log $TEMP/metadata.pagerank.directed.summary.txt 

In [63]:
if compute_pagerank:
    !cat $TEMP/metadata.pagerank.directed.summary.txt

graph loaded! It has 32514811 nodes and 110244593 edges

###Top relations:
P31	32677573
P6259	7999222
P59	7350430
P17	4867690
P684	4314512
P361	3216946
P703	3060841
P279	2929865
P171	2901186
P105	2889571

###PageRank
Max pageranks
3175	Q4167836	0.034585
82461	Q1264450	0.011703
340636	Q35252665	0.009475
1958	Q11266439	0.008330
577016	Q17442446	0.008110


In [64]:
if compute_pagerank:
    !$kgtk graph-statistics -i "$ITEMS" -o $OUT/metadata.pagerank.undirected.tsv.gz \
    --page-rank-property undirected_pagerank \
    --pagerank --statistics-only --undirected \
    --log $TEMP/metadata.pagerank.undirected.summary.txt 

Taking a look at the created file

In [65]:
if compute_pagerank:
    !cat $TEMP/metadata.pagerank.undirected.summary.txt 

graph loaded! It has 32514811 nodes and 110244593 edges

###Top relations:
P31	32677573
P6259	7999222
P59	7350430
P17	4867690
P684	4314512
P361	3216946
P703	3060841
P279	2929865
P171	2901186
P105	2889571

###PageRank
Max pageranks
82461	Q1264450	0.038235
3175	Q4167836	0.035878
7448	Q4167410	0.016248
14223	Q523	0.015989
6893	Q11173	0.013249


## Compute Degrees

Kypher can compute the out degree by counting the node2s for each node1

In [66]:
!$kypher -i "$CLAIMS" -o $TEMP/metadata.out_degree.tsv.gz \
--match '(n1)-[l]->()' \
--return 'distinct n1 as node1, count(distinct l) as node2, "out_degree" as label' 

[2021-04-20 11:51:49 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_1_c1."node1" "_aLias.node1", count(DISTINCT graph_1_c1."id") "_aLias.node2", ? "_aLias.label"
     FROM graph_1 AS graph_1_c1
     GROUP BY "_aLias.node1"
  PARAS: ['out_degree']
---------------------------------------------


In [67]:
!$kgtk add-id --id-style wikidata -i $TEMP/metadata.out_degree.tsv.gz \
/ sort -o $OUT/metadata.out_degree.tsv.gz

In [20]:
!zcat < $OUT/metadata.out_degree.tsv.gz | head

node1	node2	label	id
P10	17	out_degree	P10-out_degree-452354
P1000	8	out_degree	P1000-out_degree-2c6242
P1001	28	out_degree	P1001-out_degree-59e197
P1002	9	out_degree	P1002-out_degree-19581e
P1003	18	out_degree	P1003-out_degree-4ec959
P1004	33	out_degree	P1004-out_degree-c6f3ac
P1005	22	out_degree	P1005-out_degree-785f3e
P1006	25	out_degree	P1006-out_degree-b7a568
P1007	18	out_degree	P1007-out_degree-4ec959

gzip: stdout: Broken pipe


To count the in-degree we only care when the node2 is a wikibase-item

In [11]:
# BUG in kypher, sometimes the following command will not work, as in we'll see multilple rows for a Qnode, which is
# fixable by deleting cache
!$kypher -i "$CLAIMS" -o $TEMP/metadata.in_degree_1.tsv.gz \
--match '()-[l]->(n2 {`wikidatatype`:"wikibase-item"})' \
--return 'distinct n2 as node1, count(distinct l) as node2, "in_degree" as label' \
--order-by 'n2'

[2021-04-22 12:25:04 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_1_c1."node2" "_aLias.node1", count(DISTINCT graph_1_c1."id") "_aLias.node2", ? "_aLias.label"
     FROM graph_1 AS graph_1_c1
     WHERE graph_1_c1."node2;wikidatatype"=?
     GROUP BY "_aLias.node1"
     ORDER BY graph_1_c1."node2" ASC
  PARAS: ['in_degree', 'wikibase-item']
---------------------------------------------


The above query calculates only for indegree greater than or equal to one. Because some of the q-nodes do not appear in node 2 at all. These are the nodes that have indegree as 0 and are present in the claims file but do not appear in the above generated file.  
Hence, we filter on node1 to get a list of q-nodes that have indegree 0.

In [12]:

!kgtk ifnotexists --input-file "$CLAIMS" \
--filter-on $TEMP/metadata.in_degree_1.tsv.gz\
--input-keys node1 \
--filter-keys node1 \
-o $TEMP/indegree.zero_1.tsv.gz

In [13]:
!zcat < $TEMP/indegree.zero_1.tsv.gz | head

id	node1	label	node2	node2;wikidatatype	rank
P10-P1628-32b85d-7927ece6-0	P10	P1628	"http://www.w3.org/2006/vcard/ns#Video"	url	normal
P10-P1628-acf60d-b8950832-0	P10	P1628	"https://schema.org/video"	url	normal
P10-P1629-Q34508-bcc39400-0	P10	P1629	Q34508	wikibase-item	normal
P10-P1659-P1651-c4068028-0	P10	P1659	P1651	wikibase-property	normal
P10-P1659-P18-5e4b9c4f-0	P10	P1659	P18	wikibase-property	normal
P10-P1659-P4238-d21d1ac0-0	P10	P1659	P4238	wikibase-property	normal
P10-P1659-P51-86aca4c5-0	P10	P1659	P51	wikibase-property	normal
P10-P1855-Q15075950-7eff6d65-0	P10	P1855	Q15075950	wikibase-item	normal
P10-P1855-Q4504-a69d2c73-0	P10	P1855	Q4504	wikibase-item	normal

gzip: stdout: Broken pipe


Properties are also included. We only care about wikibase items. These are the Q-Nodes. We will also remove duplicates by using distinct.

In [15]:

!$kypher -i $TEMP/indegree.zero_1.tsv.gz --match '(n1)-[]->(n2)' \
--where 'substr(n1,1,1) >= "Q"'\
-o $TEMP/indegree.zero.tsv.gz \
--return 'distinct n1 as node1, "in_degree" as label, "0" as node2'

[2021-04-22 13:21:09 sqlstore]: IMPORT graph directly into table graph_15 from /data/hrathod/output/subset/novartis/temp.useful_files/indegree.zero_1.tsv.gz ...
[2021-04-22 13:28:13 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_15_c1."node1" "_aLias.node1", ? "_aLias.label", ? "_aLias.node2"
     FROM graph_15 AS graph_15_c1
     WHERE (substr(graph_15_c1."node1", ?, ?) >= ?)
  PARAS: ['in_degree', '0', 1, 1, 'Q']
---------------------------------------------


In [16]:
#Taking a look at the file.
!zcat < $TEMP/indegree.zero.tsv.gz | head

node1	label	node2
Q10000	in_degree	0
Q1000000	in_degree	0
Q100000003	in_degree	0
Q100000011	in_degree	0
Q100000014	in_degree	0
Q100000021	in_degree	0
Q100000029	in_degree	0
Q100000030	in_degree	0
Q100000033	in_degree	0

gzip: stdout: Broken pipe


In [17]:
#Concatenating the indegree 0 items with other degree items.
!kgtk cat -i $TEMP/indegree.zero.tsv.gz $TEMP/metadata.in_degree_1.tsv.gz -o $TEMP/metadata.in_degree.tsv.gz

In [20]:
#Adding the ID for the file
!$kgtk add-id --id-style wikidata -i $TEMP/metadata.in_degree.tsv.gz \
/ sort -o $OUT/metadata.in_degree.tsv.gz

In [21]:
!zcat < $OUT/metadata.in_degree.tsv.gz | head | col

node1	label	node2	id
Q1	in_degree	72	Q1-in_degree-872261
Q100	in_degree	4390	Q100-in_degree-c3c41d
Q1000	in_degree	1824	Q1000-in_degree-2ced18

gzip: Q10000	in_degree	0	Q10000-in_degree-5feceb
Q1000000	in_degree	0	Q1000000-in_degree-5feceb
stdout: Broken pipe
Q10000000	in_degree	1	Q10000000-in_degree-6b86b2
Q100000003	in_degree	0	Q100000003-in_degree-5feceb
Q100000011	in_degree	0	Q100000011-in_degree-5feceb
Q100000014	in_degree	0	Q100000014-in_degree-5feceb


Calculate the distribution so we can make a nice chart

In [27]:
!$kypher -i $OUT/metadata.in_degree.tsv.gz -o $OUT/statistics.in_degree.distribution.tsv \
--match '(n1)-[]->(n2)' \
--return 'distinct n2 as in_degree, count(distinct n1) as count, "count" as label' \
--order-by 'cast(n2, integer)' 

[2021-04-21 20:41:03 sqlstore]: IMPORT graph directly into table graph_14 from /data/hrathod/output/subset/novartis/useful_files/metadata.in_degree.tsv.gz ...
[2021-04-21 20:41:45 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_14_c1."node2" "_aLias.in_degree", count(DISTINCT graph_14_c1."node1") "_aLias.count", ? "_aLias.label"
     FROM graph_14 AS graph_14_c1
     GROUP BY "_aLias.in_degree"
     ORDER BY CAST(graph_14_c1."node2" AS integer) ASC
  PARAS: ['count']
---------------------------------------------


In [28]:
!head $OUT/statistics.in_degree.distribution.tsv | col

in_degree	count	label
0	26915708	count
1	3878549 count
2	809741	count
3	384964	count
4	146345	count
5	95533	count
6	66172	count
7	59216	count
8	36869	count


In [29]:
!$kypher -i $OUT/metadata.out_degree.tsv.gz -o $OUT/statistics.out_degree.distribution.tsv \
--match '(n1)-[]->(n2)' \
--return 'distinct n2 as out_degree, count(distinct n1) as count, "count" as label' \
--order-by 'cast(n2, integer)' 

[2021-04-21 20:42:27 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_13_c1."node2" "_aLias.out_degree", count(DISTINCT graph_13_c1."node1") "_aLias.count", ? "_aLias.label"
     FROM graph_13 AS graph_13_c1
     GROUP BY "_aLias.out_degree"
     ORDER BY CAST(graph_13_c1."node2" AS integer) ASC
  PARAS: ['count']
---------------------------------------------


Draw some charts

In [30]:
data = pd.read_csv(
    os.environ["OUT"] + "/statistics.in_degree.distribution.tsv", sep="\t"
)

alt.Chart(data).mark_circle(size=60).encode(
    x=alt.X("in_degree", scale=alt.Scale(type="log")),
    y=alt.Y("count", scale=alt.Scale(type="log"), title="count of nodes"),
    tooltip=["in_degree", "count"],
).interactive().properties(title="Distribution of In Degree")

alt.Chart(...)

In [31]:
data = pd.read_csv(
    os.environ["OUT"] + "/statistics.out_degree.distribution.tsv", sep="\t"
)

alt.Chart(data).mark_circle(size=60).encode(
    x=alt.X("out_degree", scale=alt.Scale(type="log")),
    y=alt.Y("count", scale=alt.Scale(type="log"), title="count of nodes"),
    tooltip=["out_degree", "count"],
).interactive().properties(title="Distribution of Out Degree")

alt.Chart(...)

## Summary of results

In [32]:
!ls -lh $OUT/*

-rw-r--r-- 1 hrathod div22  84M Apr 19 10:07 /data/hrathod/output/subset/novartis/useful_files/aliases.en.tsv.gz
-rw-r--r-- 1 hrathod div22 6.8G Apr 20 03:42 /data/hrathod/output/subset/novartis/useful_files/derived.isastar.tsv.gz
-rw-r--r-- 1 hrathod div22 108M Apr 19 15:49 /data/hrathod/output/subset/novartis/useful_files/derived.isa.tsv.gz
-rw-r--r-- 1 hrathod div22 478M Apr 19 14:45 /data/hrathod/output/subset/novartis/useful_files/derived.P279star.tsv.gz
-rw-r--r-- 1 hrathod div22  36M Apr 19 10:20 /data/hrathod/output/subset/novartis/useful_files/derived.P279.tsv.gz
-rw-r--r-- 1 hrathod div22 6.8G Apr 20 10:15 /data/hrathod/output/subset/novartis/useful_files/derived.P31P279star.tsv.gz
-rw-r--r-- 1 hrathod div22 395M Apr 19 10:19 /data/hrathod/output/subset/novartis/useful_files/derived.P31.tsv.gz
-rw-r--r-- 1 hrathod div22 160M Apr 19 10:11 /data/hrathod/output/subset/novartis/useful_files/descriptions.en.tsv.gz
-rw-r--r-- 1 hrathod div22 358M Apr 19 10:06 /data/hrathod/output/s

Highest page rank

In [25]:
if compute_pagerank:
    !$kypher -i $OUT/metadata.pagerank.undirected.tsv.gz -i "$LABELS" \
    --match 'pagerank: (n1)-[:undirected_pagerank]->(page_rank), label: (n1)-[:label]->(label)' \
    --return 'distinct n1, label as label, page_rank as undirected_page_rank' \
    --order-by 'page_rank desc' \
    --limit 10 

[2021-04-22 19:37:20 sqlstore]: IMPORT graph directly into table graph_16 from /data/hrathod/output/subset/novartis/useful_files/metadata.pagerank.undirected.tsv.gz ...
[2021-04-22 19:40:29 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_16_c1."node1", graph_9_c2."node2" "_aLias.label", graph_16_c1."node2" "_aLias.undirected_page_rank"
     FROM graph_16 AS graph_16_c1, graph_9 AS graph_9_c2
     WHERE graph_16_c1."label"=?
     AND graph_9_c2."label"=?
     AND graph_16_c1."node1"=graph_9_c2."node1"
     ORDER BY graph_16_c1."node2" DESC
     LIMIT ?
  PARAS: ['undirected_pagerank', 'label', 10]
---------------------------------------------
[2021-04-22 19:40:29 sqlstore]: CREATE INDEX on table graph_16 column node1 ...
[2021-04-22 19:41:48 sqlstore]: ANALYZE INDEX on table graph_16 column node1 ...
[2021-04-22 19:41:55 sqlstore]: CREATE INDEX on table graph_16 column label ...
[2021-04-22 19:43:34 sqlstore]: ANALYZE INDEX on table graph_1

In [34]:
!ls -l $LABELS

-rw-r--r-- 1 hrathod div22 376402061 Apr 19 00:51 /data/hrathod/output/subset/novartis/parts/labels.en.tsv.gz
